In [2]:
from selenium import webdriver
import time
import os
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
simp_path = '.env'
abs_path = os.path.abspath(simp_path)
CREDENTIAL={'EMAIL':'','PASSWORD':''}
with open(abs_path,"r") as archivo:
    for linea in archivo:        
        if (str(linea.strip('\n')).split('='))[0] == "EMAIL":
            CREDENTIAL["EMAIL"]=(str(linea.strip('\n')).split('='))[1]
        if (str(linea.strip('\n')).split('='))[0] == "PASSWORD":
            CREDENTIAL["PASSWORD"]=(str(linea.strip('\n')).split('='))[1]

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#abrir formulario
driver.get('https://sivireno.undc.edu.pe/index_login.php')
driver.find_element(by=By.XPATH, value='/html/body/div[1]/div/div/form/div[1]/a').click()
driver.find_element(by=By.XPATH, value='//*[@id="identifierId"]').send_keys(CREDENTIAL["EMAIL"])
driver.find_element(by=By.XPATH, value='//*[@id="identifierNext"]/div/button/span').click()
time.sleep(2)
driver.find_element(by=By.XPATH, value='//*[@id="password"]/div[1]/div/div[1]/input').send_keys(CREDENTIAL["PASSWORD"])
driver.find_element(by=By.XPATH, value='//*[@id="passwordNext"]/div/button/span').click()
time.sleep(2)

In [2]:
url="https://sivireno.undc.edu.pe/index_home.php?s=frm_detallelectiva.php"
driver.get(url)

In [5]:
##SCRIPT EXTRAER TODOS LOS LINK DE CARGA LECTIVA Y NO LECTIVA
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
import re
from selenium.common.exceptions import NoSuchElementException
import time
asig = []
ciclo = []
t = []
g = []
c = []
docen = []
cdl = []

baseURL="https://sivireno.undc.edu.pe/"
##la cantidad que te muestra el sivireno
cantlist=24

###########################################CREACION DE METODOS#################################################################

def addLinkExcel():
    df = pd.DataFrame({ 'ASIGNATURA': asig,'CICLO': ciclo,'TURNO': t,'GRUPO': g,'CANT': c,'DOCENTE': docen,'CDL': cdl})
    df = df[['ASIGNATURA', 'CICLO','TURNO','GRUPO','CANT','DOCENTE','CDL']]
    writer = pd.ExcelWriter('EXCEL_GENERADOS/CREATE_DET_CURSOS_GROUP'+time.strftime("%Y%m%d-%H-%M-%S")+'.xlsx')
    df.to_excel(writer, 'Sheet', index=False)
    #writer.save()
    writer.close()
###########################################CREACION DE METODOS#################################################################
try:
    for j in range(int(cantlist)):
            
            asig.append(driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/section/div/table/tbody/tr['+str(j+1)+']/td[3]').text)
                
            ciclo.append(driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/section/div/table/tbody/tr['+str(j+1)+']/td[4]').text)
        
            t.append(driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/section/div/table/tbody/tr['+str(j+1)+']/td[5]').text)
        
            g.append(driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/section/div/table/tbody/tr['+str(j+1)+']/td[6]').text)
        
            c.append(driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/section/div/table/tbody/tr['+str(j+1)+']/td[7]').text)
            
            docen.append(driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/section/div/table/tbody/tr['+str(j+1)+']/td[8]').text)
            
            cdl.append(baseURL+driver.execute_script('return document.querySelector("body > div.wrapper > div.content-wrapper.pt-2 > section > div > table > tbody > tr:nth-child('+str(j+1)+') > td:nth-child(9) > a").getAttribute("href");'))
        
                                                        
    addLinkExcel()
except:
    addLinkExcel()


In [5]:
#SCRIPT CARGA DET LECTIVA DE MANERA MASIVA
import pandas as pd
import datetime
df = pd.read_excel('cargadetlectiva-masivo.xlsx')
df
semanas_xpath= {'1':'//*[@id="exampleFormControlSelect1"]/option[1]',
               '2':'//*[@id="exampleFormControlSelect1"]/option[2]',
               '3':'//*[@id="exampleFormControlSelect1"]/option[3]',
               '4':'//*[@id="exampleFormControlSelect1"]/option[4]',
               '5':'//*[@id="exampleFormControlSelect1"]/option[5]',
               '6':'//*[@id="exampleFormControlSelect1"]/option[6]',
               '7':'//*[@id="exampleFormControlSelect1"]/option[7]',
               '8':'//*[@id="exampleFormControlSelect1"]/option[8]',
               '9':'//*[@id="exampleFormControlSelect1"]/option[9]',
               '10':'//*[@id="exampleFormControlSelect1"]/option[10]',
               '11':'//*[@id="exampleFormControlSelect1"]/option[11]',
               '12':'//*[@id="exampleFormControlSelect1"]/option[12]',
               '13':'//*[@id="exampleFormControlSelect1"]/option[13]',
               '14':'//*[@id="exampleFormControlSelect1"]/option[14]',
               '15':'//*[@id="exampleFormControlSelect1"]/option[15]',
               '16':'//*[@id="exampleFormControlSelect1"]/option[16]',}

#INICIO DE CLASES
w_iclass={'LUNES':'29/08/2022',
               'MARTES':'30/08/2022',
               'MIERCOLES':'31/08/2022',
               'JUEVES':'01/09/2022',
               'VIERNES':'02/09/2022',
               'SABADO':'03/09/2022',
               'DOMINGO':'04/09/2022'}

urltemp=""
############################################            METODOS
    #ALGORITMO PARA LA FECHA
def convertDateWrite(fecha):
    f = str(fecha).split('-')
    año = f[0]
    mes = f[1]
    d= f[2].split()
    dia = d[0]
    return dia+mes+año
    
    #ALGORTIMO HORA
def convertHourWrite(hour):
    h = str(hour).split(':')
    return h[0]+h[1]

    # 7 + W, CON RESPECTO AL DIA
def sumWeek(dia,sem):
    return convertDateWrite(datetime.datetime.strptime(w_iclass[dia], '%d/%m/%Y')+(datetime.timedelta(days=7)*(sem-1)))

def writeForm(semi,semf,dia,doc,horae,horas,asig,turno,grupo,ciclo):
    x=1
    for j in range(int(semi),int(semf+1)):
        print(str(x)+") Asig: "+asig+" "+ciclo+" "+turno+" "+grupo+" Docente :"+doc+" SEM:"+str(j)+" semi:"+str(semi)+" semf:"+str(semf)+" FECHA:"+sumWeek(dia,j))
        time.sleep(0.140)
        #ingresar semana
        driver.find_element(by=By.XPATH, value=semanas_xpath[str(int(j))]).click()
        time.sleep(0.150)
        #ingresar fecha
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_docentes"]/div[2]/div[2]/table/tbody/tr[2]/td[2]/input').send_keys(sumWeek(dia,j))
                                                
        #ingresar fecha entrada
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_docentes"]/div[2]/div[2]/table/tbody/tr[2]/td[3]/input').send_keys(str(convertHourWrite(horae)))
        #print(str(convertHourWrite(horae)))
        #ingresar fecha salida
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_docentes"]/div[2]/div[2]/table/tbody/tr[2]/td[4]/input').send_keys(str(convertHourWrite(horas)))

        #enviar registro
        driver.find_element(by=By.XPATH, value='//*[@id="asistencia_docentes"]/div[2]/div[2]/table/tbody/tr[2]/td[7]/button').click()
        time.sleep(0.100)
        x=x+1
        
############################################            METODOS/*

for row, datos in df.iterrows():
    url = datos['URL']
    semi = datos['SEM_I']
    semf = datos['SEM_F']
    dia = datos['DIA']
    doc=datos['DOCENTE']
    horae = datos['HORAE']
    horas = datos['HORAS']
    
    
    asig=datos['ASIGNATURA']
    turno=datos['TURNO']
    grupo=datos['GRUPO']
    ciclo=datos['CICLO']
    if urltemp==url:
        writeForm(semi,semf,dia,doc,horae,horas,asig,turno,grupo,ciclo)
        print("")
    else:
        if str(semi)!="nan":   
            urltemp=url
            time.sleep(2)
            driver.get(url)
            time.sleep(1.5)
            writeForm(semi,semf,dia,doc,horae,horas,asig,turno,grupo,ciclo)
            print("")
    
            

1) Asig: CULTURA AMBIENTAL II M B Docente :VEGA CANALES FELIPE SEM:2 semi:2.0 semf:5.0 FECHA:05092022
2) Asig: CULTURA AMBIENTAL II M B Docente :VEGA CANALES FELIPE SEM:3 semi:2.0 semf:5.0 FECHA:12092022
3) Asig: CULTURA AMBIENTAL II M B Docente :VEGA CANALES FELIPE SEM:4 semi:2.0 semf:5.0 FECHA:19092022
4) Asig: CULTURA AMBIENTAL II M B Docente :VEGA CANALES FELIPE SEM:5 semi:2.0 semf:5.0 FECHA:26092022

1) Asig: FÍSICA II M B Docente :VILCA ARANA MIRIAM SEM:1 semi:1.0 semf:5.0 FECHA:02092022
2) Asig: FÍSICA II M B Docente :VILCA ARANA MIRIAM SEM:2 semi:1.0 semf:5.0 FECHA:09092022
3) Asig: FÍSICA II M B Docente :VILCA ARANA MIRIAM SEM:3 semi:1.0 semf:5.0 FECHA:16092022
4) Asig: FÍSICA II M B Docente :VILCA ARANA MIRIAM SEM:4 semi:1.0 semf:5.0 FECHA:23092022
5) Asig: FÍSICA II M B Docente :VILCA ARANA MIRIAM SEM:5 semi:1.0 semf:5.0 FECHA:30092022

1) Asig: QUÍMICA GENERAL II M B Docente :VILCA ARANA MIRIAM SEM:1 semi:1.0 semf:5.0 FECHA:03092022
2) Asig: QUÍMICA GENERAL II M B Docente :